In [5]:
import re
import numpy as np
from random import randint
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

In [6]:
def data_preparation_1(in_file):
    in_file_read = open(in_file, 'r', encoding='utf-8', errors='ignore')
    for line in in_file_read:
        # remove the line/conversation codes at start of each line
        yield re.sub('^L[0-9]+\s[+$]+\s[a-z0-9]+\s[+$]+\s[a-z0-9]+\s[+$]+\s[A-Z]+\s[+$]+\s', '', line)
    finally:
        in_file_read.close()

def data_cleaner_new(in_file, out_file):
    line_generator = data_preparation_1(in_file)
        

def data_cleaner(in_file, out_file, cap_dict = 'capitalized_words.txt'):
    in_file_read = open(in_file, 'r', encoding='utf-8', errors='ignore')
    out_file_write = open(out_file, 'w')
    cap_set = set()
    for line in in_file_read:
        # remove the line/conversation codes at start of each line
        new_line = re.sub('^L[0-9]+\s[+$]+\s[a-z0-9]+\s[+$]+\s[a-z0-9]+\s[+$]+\s[A-Z]+\s[+$]+\s', '', line)
        
        new_line = new_line.lower()
        new_line = re.sub('\'m', ' am', new_line)
        new_line = re.sub('`m', ' am', new_line)
        new_line = re.sub('let\'s', 'let us', new_line)
        new_line = re.sub('let`s', 'let us', new_line)
        new_line = re.sub('\'s', ' is', new_line)
        new_line = re.sub('`s', ' is', new_line)
        new_line = re.sub('\'ll', ' will', new_line)
        new_line = re.sub('`ll', ' will', new_line)
        new_line = re.sub('\'ve', ' have', new_line)
        new_line = re.sub('`ve', ' have', new_line)
        new_line = re.sub('\'re', ' are', new_line)
        new_line = re.sub('`re', ' are', new_line)
        new_line = re.sub('\'d', ' would', new_line)
        new_line = re.sub('`d', ' would', new_line)
        new_line = re.sub('won\'t', 'will not', new_line)
        new_line = re.sub('won`t', 'will not', new_line)
        new_line = re.sub('wouldn\'t', 'would not', new_line)
        new_line = re.sub('wouldn`t', 'would not', new_line)
        new_line = re.sub('don\'t', 'do not', new_line)
        new_line = re.sub('don`t', 'do not', new_line)
        new_line = re.sub('can\'t', 'cannot', new_line)
        new_line = re.sub('can`t', 'cannot', new_line)
        new_line = re.sub(' +', ' ', new_line)
        new_line = re.sub('[-+]?[0-9]+[,0-9]*(\.[0-9]+)?', 'number', new_line)
        new_line = re.sub('[()"#@/;:<>{}+=~§|.!?,\[\]]', '', new_line)
        out_file_write.write(new_line)

    in_file_read.close()
    out_file_write.close()
    

def tokenize_file(in_file):
    in_file_read = open(in_file, 'r')
    pattern = re.compile(r'[A-Za-z]+[\w^\']*|[\w^\']*[A-Za-z]+[\w^\']*')
    all_tokens = set()
    for line in in_file_read:
        new_tokens = pattern.findall(line)
        all_tokens.update(new_tokens)
    
    in_file_read.close()
    return list(all_tokens)

def tokenize(text):
    # obtains tokens with a least 1 alphabetic character
    pattern = re.compile(r'[A-Za-z]+[\w^\']*|[\w^\']*[A-Za-z]+[\w^\']*')
    return pattern.findall(text.lower())    

def mapping(tokens):
    word_to_id = dict()
    id_to_word = dict()

    for i, token in enumerate(set(tokens)):
        word_to_id[token] = i
        id_to_word[i] = token

    return word_to_id, id_to_word

def generate_training_data(tokens, word_to_id, window_size):
    N = len(tokens)
    X, Y = [], []

    for i in range(N):
        nbr_inds = list(range(max(0, i - window_size), i)) + \
                   list(range(i + 1, min(N, i + window_size + 1)))
        for j in nbr_inds:
            X.append(word_to_id[tokens[i]])
            Y.append(word_to_id[tokens[j]])
            
    X = np.array(X)
    X = np.expand_dims(X, axis=0)
    Y = np.array(Y)
    Y = np.expand_dims(Y, axis=0)
            
    return X, Y

SyntaxError: invalid syntax (<ipython-input-6-e1bb3fda23d9>, line 2)

In [3]:
data_cleaner('movie_lines.txt', 'out.txt')

In [4]:
tokens = tokenize_file('out.txt')
word_to_id, id_to_word = mapping(tokens)
X, Y = generate_training_data(tokens, word_to_id, 3)
print(np.shape(X))
print(np.shape(Y))
vocab_size = len(id_to_word)

(1, 308310)
(1, 308310)
